In [1]:
import pandas as pd
from result_df import find_anomaly_problem_chain
from log_to_df import logs_to_dataframe
from chain_with_transformer import find_anomaly_problem_chain_transform

D:\AtomicHack-Log-Monitor\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка датасета аномалий и проблем
anomalies_problems = pd.read_csv('../Validation_Cases/ValidationCase_1/anomalies_problems.csv', sep=';')

In [3]:
anomalies_problems.head()

,ID аномалии,Аномалия,ID проблемы,Проблема
0,1,ata1: COMRESET failed,1,RAID controller firmware fault detected
1,2,RAID array degraded,1,RAID controller firmware fault detected
2,3,RAID disk offline,1,RAID controller firmware fault detected
3,4,ata1: read timeout,1,RAID controller firmware fault detected
4,5,Disk status unknown,1,RAID controller firmware fault detected


In [4]:
anomalies_problems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID аномалии  500 non-null    int64 
 1   Аномалия     500 non-null    object
 2   ID проблемы  500 non-null    int64 
 3   Проблема     500 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [5]:
print(f'Количество уникальных значений аномалий text: {len(anomalies_problems['Аномалия'].unique())}')
print(f'Количество уникальных значений аномалий id: {len(anomalies_problems['ID аномалии'].unique())}')
print(f'Количество уникальных значений проблем text: {len(anomalies_problems['Проблема'].unique())}')
print(f'Количество уникальных значений проблем id: {len(anomalies_problems['ID проблемы'].unique())}')

Количество уникальных значений аномалий text: 399
Количество уникальных значений аномалий id: 410
Количество уникальных значений проблем text: 50
Количество уникальных значений проблем id: 50


In [6]:
# Проверка на дубликаты
all_duplicates = anomalies_problems.duplicated(keep=False)
duplicate_rows = anomalies_problems[all_duplicates]
len(duplicate_rows)

0

In [7]:
# Количество аномалий к каждой проблеме
ap_group_by_problem = anomalies_problems.groupby('ID проблемы').agg({'ID аномалии': 'count'})
ap_group_by_problem

,ID аномалии
ID проблемы,
1,10
2,10
3,10
4,10
5,10
6,10
7,10
8,10
9,10


In [8]:
# Количество проблем к каждой аномалии
ap_group_by_anomalies = anomalies_problems.groupby('ID аномалии').agg({'ID проблемы': 'count'})
ap_group_by_anomalies[ap_group_by_anomalies['ID проблемы']>1]

,ID проблемы
ID аномалии,
481,10
482,10
483,10
484,10
485,10
486,10
487,10
488,10
489,10


In [9]:
anomalies_problems[anomalies_problems['ID аномалии']==481]

,ID аномалии,Аномалия,ID проблемы,Проблема
8,481,segfault at 0x0 ip 0x7f1234567890,1,RAID controller firmware fault detected
58,481,segfault at 0x0 ip 0x7f1234567890,6,Motherboard VRM failure detected
108,481,segfault at 0x0 ip 0x7f1234567890,11,Syntax error in module initialization
158,481,segfault at 0x0 ip 0x7f1234567890,16,Software update installation failed
208,481,segfault at 0x0 ip 0x7f1234567890,21,Network interface eth0 link failure
258,481,segfault at 0x0 ip 0x7f1234567890,26,SSL certificate expired
308,481,segfault at 0x0 ip 0x7f1234567890,31,Database instance stopped unexpectedly
358,481,segfault at 0x0 ip 0x7f1234567890,36,Replication lag exceeded threshold
408,481,segfault at 0x0 ip 0x7f1234567890,41,Data center cooling system failure
458,481,segfault at 0x0 ip 0x7f1234567890,46,Virtual network misconfiguration


**!!!Вопрос что делать с аномалиями, которые относятся к нескольким проблемам!!!**

In [10]:
# Нужные логи кейса 1
VC1 = logs_to_dataframe('../Validation_Cases/ValidationCase_1')
VC1.head()

,datetime,level,source,text,filename,line_number
0,2025-10-03T00:18:00,ERROR,hardware,Network interface eth0 link failure,web_server_log.txt,39601
1,2025-10-03T02:29:31,WARNING,network,segfault at 0x0 ip 0x7f1234567890,web_server_log.txt,47492


In [11]:
# Нужные логи кейса 2
VC2 = logs_to_dataframe('../Validation_Cases/ValidationCase_2')
VC2.head()

,datetime,level,source,text,filename,line_number
0,2025-10-06T01:55:34,WARNING,database,Slow query from index issue,backup_server_log.txt,6935
1,2025-10-06T05:44:04,WARNING,database,Query timeout warning,router_log.txt,20645
2,2025-10-06T05:09:49,WARNING,database,Application crash on index,storage_system_log.txt,18590
3,2025-10-06T07:42:23,ERROR,database,Index corruption on table,web_server_log.txt,27744


In [12]:
# Нужные логи кейса 3
VC3 = logs_to_dataframe('../Validation_Cases/ValidationCase_3')
VC3.head(10)

,datetime,level,source,text,filename,line_number
0,2025-10-06T00:24:39,WARNING,network,Performance degradation detected,app_server1_log.txt,1480
1,2025-10-06T08:59:32,ERROR,os,Host overheating alert,app_server1_log.txt,32373
2,2025-10-06T08:36:41,WARNING,os,Overheating alert in host,app_server2_log.txt,31002
3,2025-10-06T09:16:28,WARNING,os,Overheating alert in host,app_server2_log.txt,33389
4,2025-10-06T08:51:26,WARNING,network,Slow virtual network performance,backup_server_log.txt,31887
5,2025-10-06T02:25:21,WARNING,network,User-facing VM network issue,firewall_log.txt,8722
6,2025-10-06T11:41:12,ERROR,network,Virtual network misconfiguration,switch1_log.txt,42073


In [13]:
# Выходной датафрейм
result_df = find_anomaly_problem_chain(VC3, anomalies_problems)
result_df.head()


,ID аномалии,ID проблемы,Файл с проблемой,№ строки,Строка из лога
0,482,1,switch1_log.txt,42073,Virtual network misconfiguration
1,387,49,app_server1_log.txt,32373,Host overheating alert
2,387,49,app_server1_log.txt,32373,Host overheating alert
3,364,46,switch1_log.txt,42073,Virtual network misconfiguration
4,370,46,switch1_log.txt,42073,Virtual network misconfiguration


In [14]:
# Выходной датафрейм
result_df_t = find_anomaly_problem_chain_transform(VC3, anomalies_problems)
result_df_t.head()

,ID аномалии,Аномалия,Уверенность,ID проблемы,Файл с проблемой,№ строки,Строка из лога
0,482,Performance degradation detected,1.0,46,switch1_log.txt,42073,Virtual network misconfiguration
1,387,Overheating alert in host,1.0,49,app_server1_log.txt,32373,Host overheating alert
2,387,Overheating alert in host,1.0,49,app_server1_log.txt,32373,Host overheating alert
3,364,Slow virtual network performance,1.0,46,switch1_log.txt,42073,Virtual network misconfiguration
4,370,User-facing VM network issue,1.0,46,switch1_log.txt,42073,Virtual network misconfiguration
